In [46]:
%%javascript
function myFunc() {
        var s=document.createElement('div');
        s.innerHTML='Loading...';
        s.style.color='black';
        s.style.padding='20px';
        s.style.position='fixed';
        s.style.zIndex='9999';
        s.style.fontSize='3.0em';
        s.style.border='2px solid black';
        s.style.right='40px';s.style.top='40px';
        s.setAttribute('class','selector_gadget_loading');
        s.style.background='white';
        document.body.appendChild(s);
        s=document.createElement('script');
        s.setAttribute('type','text/javascript');
        s.setAttribute('src','http://localhost:8888/tree/parttwo/selectorgadget/public/js/selectorgadget.js');
        document.body.appendChild(s);
}
       
Jupyter.notebook.kernel.comm_manager.register_target('my_comm_target',
function(comm, msg) {
    comm.on_msg(function(msg) {
        myFunc();
        setTimeout(function(){
            console.log(msg['content']['data']);
            var lst = document.getElementsByClassName("sg_ignore");
            lst[1].value=msg['content']['data']['term'];
        }, 2000);
    });
    
comm.send({'foo': 1});
});

setTimeout(function(){
    var lst = document.getElementsByClassName("sg_ignore");
    JS_to_Python(lst[1].value)
    console.log(lst[1].value)
}, 10000);


function JS_to_Python(data){
    var comm = Jupyter.notebook.kernel.comm_manager.new_comm('JS_Python',
                                                         {'foo': 6})
    // Send data
    comm.send({'term': data})

    // Register a handler
    comm.on_msg(function(msg) {
        console.log(msg.content.data.foo);
    });
}

<IPython.core.display.Javascript object>

In [48]:
## import webbrowser
import os, time
import parsel
from urllib.request import urlopen
from lxml.html import fromstring, tostring
from IPython.display import IFrame
from IPython.core.display import display, HTML
from IPython.display import clear_output
from ipywidgets import widgets
from ipykernel.comm import Comm

class interactive(object):

    def __init__(self):
        clear_output()
        self.htmlsample = ''

    def get_page(self, url):
        html = urlopen(url).read()
        dom = fromstring(html)
        dom.make_links_absolute(url)
        return dom

    def analyze(self, url):
        dom = self.get_page(url)
        self.htmlsample = tostring(dom).decode('utf-8')

    def show(self, search_term):
        newsample = self.htmlsample
        #newsample = newsample.replace('<body', '<body onload="myFunc()"')

        newpath = os.path.realpath('./modified.html')

        with open(newpath, 'w') as f:
            f.write(newsample)

        display(IFrame('./modified.html', width=900, height=1000))


website = widgets.Text(description="Website to scrape", width=200)
display(website)

search_term = widgets.Text(description="Selector", width=200)
display(search_term)

button = widgets.Button(description="Check the selector")
display(button)

def on_button_clicked(b):
    obj = interactive()
    obj.analyze(str(website.value))
    obj.show(search_term)
    my_comm = Comm(target_name='my_comm_target', data={'foo': 1})
    my_comm.send({'term': search_term.value})
    get_ipython().kernel.comm_manager.register_target('JS_Python', target_func)

button.on_click(on_button_clicked)

def target_func(comm, msg):
    # comm is the kernel Comm instance
    # msg is the comm_open message

    # Register handler for later messages
    @comm.on_msg
    def _recv(msg):
        search_term.value = msg['content']['data']['term']

    comm.send({'foo': 5})

